# import the necessary libraries 

In [ ]:
pip install git+https://github.com/boudinfl/pke.git


In [ ]:
pip install contractions

In [ ]:
import pandas as pd
import numpy as np
import re 
import contractions
import nltk 
from nltk.tokenize import word_tokenize
import spacy
from nltk.corpus import stopwords
import pke 

# Reading the Article

In [ ]:
file_f = open('/content/AI.txt','r',errors='ignore')
text=file_f.read()
print(text)
file_f.close()

# Text preprocessing 

In [ ]:
string_1 = re.sub(r'[\[]\d\d+[\]]','',text)
string_1

In [ ]:
All_punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
All_punct=All_punct.replace(".","")
All_punct=All_punct.replace(",","")
for i in string_1:
    if i in All_punct:
        string_1 = string_1.replace(i, "")
print(string_1)

In [ ]:
string_2 = re.sub(r'\d+','',string_1)
string_2 

In [ ]:
# remove arabic words 
string_3 = re.sub(r"[ا-ي]",'',string_2)
string_3 

In [ ]:
#remove emails
string_4 = re.sub(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",'',string_3)
string_4 


In [ ]:
# remove URL 
string_5 = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})",'',string_4)
string_5 

In [ ]:
string_5.lower()

In [ ]:
fixed_word = []
for w in string_5.split() : 
    fixed_word.append(contractions.fix(w))
    
text_book = ' '.join(fixed_word)    


In [ ]:
text_book

# Key phrases extraction (using 5 algorithms)

In [ ]:
# 1 algorithm (topic rank) graph 
extractor = pke.unsupervised.TopicRank()

# load the content of the document, here document is expected to be a simple 
# test string and preprocessing is carried out using spacy
extractor.load_document(input=text_book, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases = extractor.get_n_best(n=15)

In [ ]:
keyphrases 

In [ ]:
# 2 algorithm (YAKE) statistical 
from pke.lang import stopwords

# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
stoplist = stopwords.get('english')
extractor.load_document(input=text_book,
                        language='en',
                        stoplist=stoplist,
                        normalization=None)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8 
keyphrases_1 = extractor.get_n_best(n=15, threshold=threshold)

In [ ]:
# 3 algortihm (Text Rank) graph 
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_book,
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases_2 = extractor.get_n_best(n=15)

In [ ]:
# 4 algorithm (single rank) graph 
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a SingleRank extractor.
extractor = pke.unsupervised.TfIdf()

# 2. load the content of the document.
extractor.load_document(input=text_book,
                        language='en',
                        normalization=None)

# 3. select the longest sequences of nouns and adjectives as candidates.
extractor.candidate_selection()

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk. In the graph, nodes are words of
#    certain part-of-speech (nouns and adjectives) that are connected if
#    they occur in a window of 10 words.
extractor.candidate_weighting()

# 5. get the 10-highest scored candidates as keyphrases
keyphrases_3 = extractor.get_n_best(n=15)

In [ ]:
# 5 algortihm (position rank) graph 
# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN', 'ADJ'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor.load_document(input=text_book,
                        language='en',
                        normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases_4 = extractor.get_n_best(n=15)

In [ ]:
pip install rake-nltk

In [ ]:
 # 6 algortihm (rake)
from rake_nltk import Rake
nltk.download('stopwords')
nltk.download('punkt')
r = Rake()
r.extract_keywords_from_text(text_book)
rak =r.get_ranked_phrases_with_scores()[:15]

In [ ]:
rak 

In [ ]:
#1 algorithm (topic rank) graph 
print(keyphrases)
# 2 algorithm (YAKE) statistical 
print(keyphrases_1)
# 3 algortihm (Text Rank) graph
print(keyphrases_2)
# 4 algorithm (single rank) graph 
print(keyphrases_3)
# 5 algortihm (position rank) graph 
print(keyphrases_4)

In [ ]:
# 2 algorithm (YAKE) statistical 
print(keyphrases_1)

In [ ]:
# 3 algortihm (Text Rank) graph
print(keyphrases_2)

In [ ]:
keyphrases

# most common keys

In [ ]:
# most common key phrases 
common = {}

for i,(key,value) in enumerate(keyphrases):
     if common.get(key)!= None :
          common[key] = common[key] +1 
     else:
          common[key] = 1


In [ ]:
common 

In [ ]:
#common
#s=[]
#p=[]
#for i ,j in common.items():
#  if(type(i)!=tuple):
#    s.append(i)
#    p.append(j)
#p

In [ ]:
import pandas as pd
#q={}
#for index,value in zip(s,p):
#  q.update({index:value})
#q

In [ ]:
for i,(key,value) in enumerate(keyphrases_1):
     if common.get(key)!= None :
          common[key] = common[key] +1 
     else:
          common[key] = 1

#enumerate(keyphrases)

In [ ]:
keyphrases_1 

In [ ]:
common 

In [ ]:
for i,(key,value) in enumerate(keyphrases_2):
     if common.get(key)!= None :
          common[key] = common[key] +1 
     else:
          common[key] = 1

#enumerate(keyphrases)

In [ ]:
common 

In [ ]:
for i,(key,value) in enumerate(keyphrases_3):
     if common.get(key)!= None :
          common[key] = common[key] +1 
     else:
          common[key] = 1

#enumerate(keyphrases)

In [ ]:
common

In [ ]:
for i,(key,value) in enumerate(keyphrases_4):
     if common.get(key)!= None :
          common[key] = common[key] +1 
     else:
          common[key] = 1

#enumerate(keyphrases)

In [ ]:
common 

In [ ]:
import operator

In [ ]:
common = sorted(common.items(), key=operator.itemgetter(1),reverse=True)

In [ ]:
common 

In [ ]:
# common 
#t=[]
#common.keys()

In [ ]:
#for (key,value) in rak:
 #    if key in rak:
  #        common[key] += 1
   #  else:
    #      common[key] = 1
#enumerate(rak)

In [ ]:
common_list = []
for i in common :
  common_list.append(i[0])



In [ ]:
common_list=common_list[:24] 

In [ ]:
common_list 

In [ ]:
import pandas as pd


In [ ]:
#for i in common_list :
#  if type(i) != str :
#    common_list.remove(i)

In [ ]:
#common_list 

In [ ]:
# common_list.remove(178.90476190476193)
# common_list.remove(66.16666666666666)
# common_list.remove(36.5)

In [ ]:
#common_list 

In [ ]:
!pip install keybert


In [ ]:
pip install flashtext

In [ ]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

In [ ]:
#common_list

In [ ]:
#keyphrases[0][0]

In [ ]:
#keyphrasess = []
#for i in keyphrases :
#   keyphrasess = keyphrases[i][0]


In [ ]:
#keyphrasess

In [ ]:
!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en
!pip install bert-extractive-summarizer --upgrade --force-reinstall
#!pip install spacy==2.1.3 --upgrade --force-reinstall
!pip install -U nltk
!pip install -U pywsd

In [ ]:
pip install summarizer

In [ ]:
#from summarizer import summarize
#summ_text =summarize('Machine learning', text_book)

In [ ]:
#summ_text 

# sentence mapping for each keyphrase 

In [ ]:
# t3ml df feha el key we el paragraph bta3o , el paragraph e5o4 3la el t5 transform
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(text_book)
keyword_sentence_mapping = get_sentences_for_keyword(common_list, sentences)
        
print (keyword_sentence_mapping)

In [ ]:
keyword_sentence_mapping

In [ ]:
list(keyword_sentence_mapping.values())[0]

In [ ]:
pip install python-pipeline

In [ ]:
!pip install keybert


In [ ]:
#text_book

In [ ]:
#from keybert import KeyBERT
#model =KeyBERT()
#keyphrases=model.extract_keywords(text_book,top_n=5)

In [ ]:
#keyphrases

In [ ]:
pip install pipelines==0.0.12

In [ ]:
!pip install -U transformers==3.0.0

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git

In [ ]:
!pip install transformers

In [ ]:
!git clone https://github.com/amontgomerie/question_generator/

In [ ]:
%cd question_generator/
%load questiongenerator.py
from questiongenerator import QuestionGenerator
from questiongenerator import print_qa

In [ ]:

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"

In [ ]:
!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en
!pip install bert-extractive-summarizer --upgrade --force-reinstall
!pip install spacy==2.1.3 --upgrade --force-reinstall
!pip install -U nltk
!pip install -U pywsd


In [ ]:
import nltk 
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
# import requests
# import json
# import re
# import random
# from pywsd.similarity import max_similarity
# from pywsd.lesk import adapted_lesk
# from pywsd.lesk import simple_lesk
# from pywsd.lesk import cosine_lesk
# from nltk.corpus import wordnet as wn

# # Distractors from Wordnet
# def get_distractors_wordnet(syn,word):
#     distractors=[]
#     word= word.lower()
#     orig_word = word
#     if len(word.split())>0:
#         word = word.replace(" ","_")
#     hypernym = syn.hypernyms()
#     if len(hypernym) == 0: 
#         return distractors
#     for item in hypernym[0].hyponyms():
#         name = item.lemmas()[0].name()
#         #print ("name ",name, " word",orig_word)
#         if name == orig_word:
#             continue
#         name = name.replace("_"," ")
#         name = " ".join(w.capitalize() for w in name.split())
#         if name is not None and name not in distractors:
#             distractors.append(name)
#     return distractors

# def get_wordsense(sent,word):
#     word= word.lower()
    
#     if len(word.split())>0:
#         word = word.replace(" ","_")
    
    
#     synsets = wn.synsets(word,'n')
#     if synsets:
#         wup = max_similarity(sent, word, 'wup', pos='n')
#         adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
#         lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
#         return synsets[lowest_index]
#     else:
#         return None

# # Distractors from http://conceptnet.io/
# def get_distractors_conceptnet(word):
#     word = word.lower()
#     original_word= word
#     if (len(word.split())>0):
#         word = word.replace(" ","_")
#     distractor_list = [] 
#     url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
#     obj = requests.get(url).json()

#     for edge in obj['edges']:
#         link = edge['end']['term'] 

#         url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
#         obj2 = requests.get(url2).json()
#         for edge in obj2['edges']:
#             word2 = edge['start']['label']
#             if word2 not in distractor_list and original_word.lower() not in word2.lower():
#                 distractor_list.append(word2)
                   
#     return distractor_list

# key_distractor_list = {}

# for keyword in keyword_sentence_mapping:
#     wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
#     if wordsense:
#         distractors = get_distractors_wordnet(wordsense,keyword)
#         if len(distractors) ==0:
#             distractors = get_distractors_conceptnet(keyword)
#         if len(distractors) != 0:
#             key_distractor_list[keyword] = distractors
#     else:
        
#         distractors = get_distractors_conceptnet(keyword)
#         if len(distractors) != 0:
#             key_distractor_list[keyword] = distractors

# index = 1
# print ("#############################################################################")
# print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
# print ("#############################################################################\n\n")
# for each in key_distractor_list:
#     sentence = keyword_sentence_mapping[each][0]
#     pattern = re.compile(each, re.IGNORECASE)
#     output = pattern.sub( " _______ ", sentence)
#     print ("%s)"%(index),output)
#     choices = [each.capitalize()] + key_distractor_list[each]
#     top4choices = choices[:4]
#     random.shuffle(top4choices)
#     optionchoices = ['a','b','c','d']
#     for idx,choice in enumerate(top4choices):
#         print ("\t",optionchoices[idx],")"," ",choice)
#     print ("\nMore options: ", choices[4:20],"\n\n")
#     index = index + 1

 

In [ ]:
keyword_sentence_mapping

In [ ]:
phrases = []
paragraph = []
for i in keyword_sentence_mapping : 
  phrases.append(i)
  paragraph.append(keyword_sentence_mapping[i][0])

In [ ]:
phrases 

In [ ]:
paragraph

In [ ]:
key_dics = {
    'Phrases':phrases,
    'Paragraphs':paragraph
}

In [ ]:
df= pd.DataFrame(key_dics)
df.head()

In [ ]:
df.to_csv('sentence_mapping.csv')

# question generator by using text 

In [ ]:
qg = QuestionGenerator()

In [ ]:
qa_list = qg.generate(
    text_book, 
    num_questions=20, 
    answer_style='all'
)
print_qa(qa_list)

In [ ]:
qa_list = qg.generate(
    text, 
    num_questions=20, 
    answer_style='all'
)
print_qa(qa_list)

In [ ]:
!pip install -U transformers==3.0.0

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git

In [ ]:
%cd question_generation

In [ ]:
from pipelines import pipeline

In [ ]:
# single_task 
nlp = pipeline("question-generation")

In [ ]:
#nlp(text_book)

In [ ]:
nlp_1 = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [ ]:
# piplines should take substring
#nlp_1(text_book)

In [ ]:
keyword_sentence_mapping.values()

In [ ]:
for i in keyword_sentence_mapping.values():
  print (i)

In [ ]:
v = list(keyword_sentence_mapping.values())

In [ ]:
#for i in v :
 # print(nlp_1(i))

In [ ]:
#qa_list = qg.generate(
 #   text, 
  #  num_questions=20, 
   # answer_style='all'
#)
#print_qa(qa_list)

In [ ]:
!pip install summarizer

In [ ]:
# used many question generators by ur dataframe at least 2 
# used 2 queastion answering by ur dataframe
# used many anserwing generators by ur dataframe at least 2  